In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


In [3]:
db, coll = connectCollection('companies','companies')

In [19]:
#query para filtrar solo las compañias relacionadas de alguna manera con design y que son posteriores a 2009.
#solamente devuelve los valores que me interesan: name, category, location, founded year e id.
new_companies = list(coll.find({'$and':
                           [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            { "offices.longitude": {"$exists": True,"$ne":None} },
                            {'deadpooled_year':None},
                            {'$or':[{'category_code': 'advertising' },
                                    {'category_code': 'software' },
                                    {'category_code': 'mobile' },
                                    {'category_code': 'web' },
                                    {'category_code': 'games_video' }]}]}, 
                          {'name':1,'founded_year': 1, '_id': 0,'offices.latitude':1, 
                           'offices.longitude':1, 'category_code':1, 'location':1 }))

In [20]:
print(len(new_companies))
new_companies[0]

4791


{'category_code': 'web',
 'founded_year': 2001,
 'name': 'TechnologyGuide',
 'offices': [{'latitude': 39.06685, 'longitude': -84.351532}]}

In [21]:
#funcion para sacar las coordenadas de las queries. Solamente saca la primera de las oficinas.
def getLocation(company):
    longitude = company['offices'][0]['longitude']
    latitude = company['offices'][0]['latitude']
    coordinates = [float(longitude), float(latitude)]
    
    return coordinates

In [22]:
getLocation(new_companies[0]) #probando la función en una company.

[-84.351532, 39.06685]

In [23]:
#funcion para crear df de la query con las coordenadas
def createDfFromMongo(query):
    df = pd.DataFrame(query)
    coords=[{'type': 'Point', 'coordinates':getLocation(q)}for q in query]
    df['office_coords']=coords
    df=df[['category_code','founded_year','name','office_coords']]
    return df
    

In [24]:
df= createDfFromMongo(new_companies)

In [17]:
df.head()

,category_code,founded_year,name,office_coords
0,web,2011,Mokitown,"{'type': 'Point', 'coordinates': [-95.712891, ..."
1,mobile,2010,GENWI,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,web,2013,Fixya,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,games_video,2011,Social Gaming Network,"{'type': 'Point', 'coordinates': [-122.161523,..."
4,mobile,2012,Skydeck,"{'type': 'Point', 'coordinates': [-122.3254705..."


In [27]:
df.to_json('df_design.json', orient='records') #guardando el df en json para Mongo.